In [4]:
import pandas as pd
import osmnx as ox
import networkx as nx
import json
import matplotlib.pyplot as plt

In [5]:
# Loading the graph back from GraphML
# "C:\Users\zliu33\Downloads\G1_copy.graphml"

G3 = ox.load_graphml(filepath=r'C:\Users\zliu33\Downloads\G1_copy_UEP.graphml')

In [6]:
gdf_nodes3, gdf_edges3 = ox.utils_graph.graph_to_gdfs(G3, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [7]:
gdf_edges4 = gdf_edges3.copy()
gdf_edges4[['CO', 'PM25', 'PM10', 'NOx_ppb']] = gdf_edges4[['CO', 'PM25', 'PM10', 'NOx_ppb']].fillna(0)

In [8]:
##### new G without nan
G4 = ox.utils_graph.graph_from_gdfs(gdf_nodes3, gdf_edges4)
gdf_nodes4, gdf_edges4 = ox.utils_graph.graph_to_gdfs(G4, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)


In [9]:
for u, v, data in G4.edges(data=True):
    # Convert CO to float, if it exists
    if 'CO' in data:
        data['CO'] = float(data['CO'])
    else:
        # Assign a default value if CO doesn't exist
        data['CO'] = float('inf')  

    # Convert PM10 to float, if it exists
    if 'PM10' in data:
        data['PM10'] = float(data['PM10'])
    else:
        # Assign a default value if PM10 doesn't exist
        data['PM10'] = float('inf')  

    # Convert PM25 to float, if it exists
    if 'PM25' in data:
        data['PM25'] = float(data['PM25'])
    else:
        # Assign a default value if PM25 doesn't exist
        data['PM25'] = float('inf')  

    # Convert NOx_ppb to float, if it exists
    if 'NOx_ppb' in data:
        data['NOx_ppb'] = float(data['NOx_ppb'])
    else:
        # Assign a default value if NOx_ppb doesn't exist
        data['NOx_ppb'] = float('inf')  


In [10]:
# "C:\Users\zliu33\Downloads\OD_lonlat.csv"

in1 = (r'C:\Users\zliu33\Downloads\OD_lonlat.csv') 
OD_pairs_weekday = pd.read_csv(in1, sep=',')

In [11]:
OD_pairs_weekday.head()

,schid,name,destination,centroid,Total Population: 5 to 9 Years,Destination_Lon,Destination_Lat,Centroid_Lon,Centroid_Lat
0,1890020,Tucker School,POINT (-71.09457196730209 42.26082173129278),POINT (-71.09606521677901 42.25433910633278),575,-71.094572,42.260822,-71.096065,42.254339
1,350154,Chittick Elementary School,POINT (-71.10478211366396 42.26725597666186),POINT (-71.1110031436925 42.26933718298657),618,-71.104782,42.267256,-71.111003,42.269337
2,350008,Ellison-Parks Early Education School,POINT (-71.0918906047409 42.271788648914985),POINT (-71.0859459425359 42.274155826818486),337,-71.091891,42.271789,-71.085946,42.274156
3,1890010,Glover School,POINT (-71.07546412515123 42.26145937586246),POINT (-71.07816920433467 42.26402484160749),446,-71.075464,42.261459,-71.078169,42.264025
4,350054,Taylor Elementary School,POINT (-71.07743056673235 42.277860739820945),POINT (-71.07461281838836 42.27386670204792),292,-71.077431,42.277861,-71.074613,42.273867


In [16]:
lat_o_lst = OD_pairs_weekday['Centroid_Lat']
lon_o_lst = OD_pairs_weekday['Centroid_Lon']
lat_d_lst = OD_pairs_weekday['Destination_Lat']
lon_d_lst = OD_pairs_weekday['Destination_Lon']
num_o_lst = ['{}_{}'.format(lat, lon) for lat, lon in zip(lat_o_lst, lon_o_lst)]
num_d_lst = OD_pairs_weekday['schid']
count_lst = OD_pairs_weekday['Total Population: 5 to 9 Years']


In [12]:
### test

In [ ]:
final_routes_dict_length = {}  # Dictionary to store routes with OD_pair_id as the key
K = 1

# Loop over each OD pair
for i in range(0, 4000):
    try:
        lat_orig = lat_o_lst[i]
        long_orig = lon_o_lst[i]
        orig_node = ox.distance.nearest_nodes(G4, long_orig, lat_orig)
    
        lat_dest = lat_d_lst[i]
        long_dest = lon_d_lst[i]
        dest_node = ox.distance.nearest_nodes(G4, long_dest, lat_dest)
        
        # Get the routes using 'length' as the weight
        routes1 = list(ox.distance.k_shortest_paths(G4, orig_node, dest_node, K, weight='length'))
        
        # If routes exist for this OD pair, add to the dictionary
        if routes1:
            od_id = f"{num_o_lst[i]}_{num_d_lst[i]}"
            
            # Initialize dictionary entry with route and placeholder for attributes
            if od_id not in final_routes_dict_length:
                final_routes_dict_length[od_id] = {
                    'volume': float(count_lst[i]),
                    'path_1': routes1[0],
                    'path_1_distance': 0,
                    'path_1_CO': 0,
                    'path_1_PM10': 0,
                    'path_1_PM25': 0,
                    'path_1_NOx': 0
                }
                
            # Calculate and update properties for route1 (based on 'length')
            for u, v in zip(routes1[0][:-1], routes1[0][1:]):
                edge_data = G4.get_edge_data(u, v, 0)  # Assuming there's only one edge between any two nodes
                if edge_data:
                    final_routes_dict_length[od_id]['path_1_distance'] += float(edge_data.get('length', 0))
                    final_routes_dict_length[od_id]['path_1_CO'] += float(edge_data.get('CO', 0))
                    final_routes_dict_length[od_id]['path_1_PM10'] += float(edge_data.get('PM10', 0))
                    final_routes_dict_length[od_id]['path_1_PM25'] += float(edge_data.get('PM25', 0))
                    final_routes_dict_length[od_id]['path_1_NOx'] += float(edge_data.get('NOx_ppb', 0))
                     
                
    except Exception as e:
        print('Exception:', e)

In [ ]:
# PM25 shortest path
final_routes_dict_PM25 = {}  # Dictionary to store routes with OD_pair_id as the key
K = 1

for i in range(0, 4000):
    try:
        lat_orig = lat_o_lst[i]
        long_orig = lon_o_lst[i]
        orig_node = ox.distance.nearest_nodes(G4, long_orig, lat_orig)
    
        lat_dest = lat_d_lst[i]
        long_dest = lon_d_lst[i]
        dest_node = ox.distance.nearest_nodes(G4, long_dest, lat_dest)
        
        # Get the routes using 'PM25' as the weight
        routes4 = list(ox.distance.k_shortest_paths(G4, orig_node, dest_node, K, weight='PM25'))
        
        # If routes exist for this OD pair, add to the dictionary
        if routes4:
            od_id = f"{num_o_lst[i]}_{num_d_lst[i]}"
            
            # Initialize dictionary entry with route and placeholder for attributes
            if od_id not in final_routes_dict_PM25:
                final_routes_dict_PM25[od_id] = {
                    'volume': float(count_lst[i]),
                    'path_4': routes4[0],
                    'path_4_distance': 0,
                    'path_4_CO': 0,
                    'path_4_PM10': 0,
                    'path_4_PM25': 0,
                    'path_4_NOx': 0
                }
                
            # Calculate and update properties for route1 (based on 'length')
            for u, v in zip(routes4[0][:-1], routes4[0][1:]):
                edge_data = G4.get_edge_data(u, v, 0)  # Assuming there's only one edge between any two nodes
                if edge_data:
                    final_routes_dict_PM25[od_id]['path_4_distance'] += float(edge_data.get('length', 0))
                    final_routes_dict_PM25[od_id]['path_4_CO'] += float(edge_data.get('CO', 0))
                    final_routes_dict_PM25[od_id]['path_4_PM10'] += float(edge_data.get('PM10', 0))
                    final_routes_dict_PM25[od_id]['path_4_PM25'] += float(edge_data.get('PM25', 0))
                    final_routes_dict_PM25[od_id]['path_4_NOx'] += float(edge_data.get('NOx_ppb', 0))
                     
                
    except Exception as e:
        print('Exception:', e)